## Imports and load of environment variables

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

PROJECT_ID = os.getenv("PROJECT_ID")
REGION = os.getenv("REGION")
MODEL_DISPLAY_NAME = os.getenv("MODEL_DISPLAY_NAME")
HF_MODEL_ID = os.getenv("HF_MODEL_ID")
HF_TASK = os.getenv("HF_TASK")
DEPLOY_IMAGE_URI = os.getenv("DEPLOY_IMAGE_URI")
MACHINE_TYPE = os.getenv("MACHINE_TYPE")
ACCELERATOR_TYPE = os.getenv("ACCELERATOR_TYPE")


##  Upload Hugging Face model to Google Vertex AI Model Registry

In [2]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    serving_container_image_uri=DEPLOY_IMAGE_URI,
    serving_container_environment_variables={
        "HF_MODEL_ID": HF_MODEL_ID,
        "HF_TASK": HF_TASK
    }
)

print(f"Model uploaded: {model.resource_name}")


c:\Users\Luis\anaconda3\envs\LLM\lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Creating Model
Create Model backing LRO: projects/198380589718/locations/us-east4/models/7506062417617485824/operations/3627800682695753728
Model created. Resource name: projects/198380589718/locations/us-east4/models/7506062417617485824@1
To use this Model in another session:
model = aiplatform.Model('projects/198380589718/locations/us-east4/models/7506062417617485824@1')
Model uploaded: projects/198380589718/locations/us-east4/models/7506062417617485824


## Create endpoint and deploy model into Vertex AI

In [ ]:
endpoint_display_name = f"{MODEL_DISPLAY_NAME}-endpoint"
endpoints = aiplatform.Endpoint.list(filter=f'display_name="{endpoint_display_name}"')

if endpoints:
    endpoint = endpoints[0]
    print(f"Using existing endpoint: {endpoint.resource_name}")
else:
    endpoint = aiplatform.Endpoint.create(display_name=endpoint_display_name)
    print(f"Created new endpoint: {endpoint.resource_name}")


deployed_model = model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=MODEL_DISPLAY_NAME,
    machine_type=MACHINE_TYPE,
    accelerator_type=ACCELERATOR_TYPE,
    accelerator_count=1,
)

print(f"Model deployed to endpoint: {endpoint.resource_name}")

Creating Endpoint
Create Endpoint backing LRO: projects/198380589718/locations/us-east4/endpoints/9008514270647812096/operations/52646266005356544
Endpoint created. Resource name: projects/198380589718/locations/us-east4/endpoints/9008514270647812096
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/198380589718/locations/us-east4/endpoints/9008514270647812096')
Created new endpoint: projects/198380589718/locations/us-east4/endpoints/9008514270647812096
Deploying model to Endpoint : projects/198380589718/locations/us-east4/endpoints/9008514270647812096
Deploy Endpoint model backing LRO: projects/198380589718/locations/us-east4/endpoints/9008514270647812096/operations/5553089523396640768
Endpoint model deployed. Resource name: projects/198380589718/locations/us-east4/endpoints/9008514270647812096
Model deployed to endpoint: projects/198380589718/locations/us-east4/endpoints/9008514270647812096


## Test model using https request

In [ ]:
import requests
import google.auth
import google.auth.transport.requests

ENDPOINT_ID = endpoint.name.split("/")[-1]

creds, project = google.auth.default()
auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)
AUTH_TOKEN = creds.token

endpoint_url = f"https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:predict"

headers = {
    "Authorization": f"Bearer {AUTH_TOKEN}",
    "Content-Type": "application/json"
}

payload = {
    "instances": [
        "[INST]Given the symptoms, explain these findings: Mild memory loss. The patient is 69 years old. Sleep apnea. Attention deficit disorder. The patient is obese and sedentary. Poor short-term memory. She has diabetes and high blood pressure.[/INST]"
    ],
    "parameters": {
        "temperature": 1.0,
        "max_new_tokens": 512,
        "top_k": 50,
        "top_p": 1.0,
        "repetition_penalty": 1.0
    }
}

try:
    response = requests.post(endpoint_url, json=payload, headers=headers)
    response.raise_for_status()
    print("Success! Prediction:")
    print(response.json())

except requests.exceptions.HTTPError as err:
    print(f"HTTP Error: {err}")
    print(f"Response Text: {err.response.text}")

Success! Prediction:
{'predictions': [[{'generated_text': '[INST]Given the symptoms, explain these findings: Mild memory loss. The patient is 69 years old. Sleep apnea. Attention deficit disorder. The patient is obese and sedentary. Poor short-term memory. She has diabetes and high blood pressure.[/INST] Based on the symptoms and medical history provided, the patient is likely experiencing a condition known as "Mild Cognitive Impairment with Seizures." This condition is characterized by mild cognitive impairment, which includes mild memory loss, poor short-term memory, and poor short-term attention, alongside the presence of seizures. The patient\'s age, weight, and lifestyle factors such as obesity and sedentary lifestyle, along with her medical history of diabetes and high blood pressure, are consistent with this condition. The combination of these factors makes it a plausible diagnosis for the symptoms described. It\'s important to consider this diagnosis when evaluating the patient

## Delete endpoint and model to prevent charges

In [ ]:
ENDPOINT_ID_TO_DELETE = endpoint.name.split("/")[-1]

aiplatform.init(project=PROJECT_ID, location=REGION)

print(f"Targeting specific endpoint for deletion: {ENDPOINT_ID_TO_DELETE}")

try:
    endpoint = aiplatform.Endpoint(endpoint_name=ENDPOINT_ID_TO_DELETE)
    
    endpoint.delete(force=True)
    
    print(f"Successfully initiated deletion for endpoint {ENDPOINT_ID_TO_DELETE}.")

    models = aiplatform.Model.list(filter=f'display_name="{MODEL_DISPLAY_NAME}"')
    
    for model in models:
        print(f"Deleting model: {model.resource_name}")
        model.delete()

except Exception as e:
    print(f"An error occurred during endpoint deletion: {e}")

Targeting specific endpoint for deletion: 9008514270647812096
Undeploying Endpoint model: projects/198380589718/locations/us-east4/endpoints/9008514270647812096
Undeploy Endpoint model backing LRO: projects/198380589718/locations/us-east4/endpoints/9008514270647812096/operations/221953464496816128
Endpoint model undeployed. Resource name: projects/198380589718/locations/us-east4/endpoints/9008514270647812096
Deleting Endpoint : projects/198380589718/locations/us-east4/endpoints/9008514270647812096
Endpoint deleted. . Resource name: projects/198380589718/locations/us-east4/endpoints/9008514270647812096
Deleting Endpoint resource: projects/198380589718/locations/us-east4/endpoints/9008514270647812096
Delete Endpoint backing LRO: projects/198380589718/locations/us-east4/operations/4833639482924204032
Endpoint resource projects/198380589718/locations/us-east4/endpoints/9008514270647812096 deleted.
Successfully initiated deletion for endpoint 9008514270647812096.
